In [61]:
#Advanced RAG with multiple data sources
!pip install arxiv



In [62]:
#Tool1
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [63]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\HARNISHA\\AppData\\Roaming\\Python\\Python312\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [64]:
#Data ingest->web based 
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://www.worldhistory.org/Mahabharata/")
web_docs=loader.load()
web_docs

[Document(metadata={'source': 'https://www.worldhistory.org/Mahabharata/', 'title': 'Mahabharata - World History Encyclopedia', 'description': 'The Mahabharata is an ancient Indian epic where the main story revolves around two branches of a family - the Pandavas and Kauravas - who, in the Kurukshetra...', 'language': 'en'}, page_content='\n\n\n\nMahabharata - World History Encyclopedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\nFacebook\n\n\n\n\n\nMembership\nNo ads\n\n\n\n\n\n\n\n\n❯\n\n\n\n\n\n \n\n\n\n\n\n\n\n\nEncyclopedia\n\nIndex\nTimeline\nMaps\nBook Reviews\nWeights & Measures\nTranslations\n\n\n\nEducation\n\nTeaching Materials\nLink Database\nQuizzes\nPrimary Sources\nCollections\nSchool Subscription\n\n\n\nMedia\n\nMedia Lib

In [65]:
#dividing into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
web_documents=text_splitter.split_documents(web_docs)
web_documents

[Document(metadata={'source': 'https://www.worldhistory.org/Mahabharata/', 'title': 'Mahabharata - World History Encyclopedia', 'description': 'The Mahabharata is an ancient Indian epic where the main story revolves around two branches of a family - the Pandavas and Kauravas - who, in the Kurukshetra...', 'language': 'en'}, page_content='Mahabharata - World History Encyclopedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\nFacebook\n\n\n\n\n\nMembership\nNo ads\n\n\n\n\n\n\n\n\n❯\n\n\n\n\n\n \n\n\n\n\n\n\n\n\nEncyclopedia\n\nIndex\nTimeline\nMaps\nBook Reviews\nWeights & Measures\nTranslations\n\n\n\nEducation\n\nTeaching Materials\nLink Database\nQuizzes\nPrimary Sources\nCollections\nSchool Subscription\n\n\n\nMedia\n\nMedia Library\nAu

In [66]:
#converting chunks -> embeddings and storing in vector database
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

import os
from dotenv import load_dotenv
load_dotenv()
os.environ['GOOGLE_API_KEY']=os.getenv("GEMINI_API_KEY")

db=FAISS.from_documents(web_documents,GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001"))



In [67]:
#Retriever (because we prefer chain retriever over similarity search)
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017DEB206E70>, search_kwargs={})

In [68]:
#take this retriever and make it a tool 
#tool2
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"Mahabharata_search", 
                                     "Search for information about Mahabharata. For any question about Mahabharata, you should use this tool")

retriever_tool.name



'Mahabharata_search'

In [69]:
# tool3
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))

In [70]:
#Combining all tools
all_tools=[wiki,retriever_tool,arxiv]
all_tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\HARNISHA\\AppData\\Roaming\\Python\\Python312\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Mahabharata_search', description='Search for information about Mahabharata. For any question about Mahabharata, you should use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000017DE83F6AC0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017DEB206E70>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_releva

In [71]:
#creating llm
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

#promt -> we are calling already built-in prompts from langchain

# from langchain import hub
# prompt =hub.pull("deanmachines-ai/superb_system_instruction_prompt")
# prompt.messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You must use tools to answer questions."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


#Agents
from langchain.agents import create_tool_calling_agent
gemini_agent=create_tool_calling_agent(llm,tools,prompt)




In [72]:
#AgentExecutor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=gemini_agent, tools=all_tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[l

In [73]:
agent_executor.invoke({"input":"Name the characters of Mahabharata"})



> Entering new AgentExecutor chain...

Invoking: `Mahabharata_search` with `{'query': 'characters of Mahabharata'}`


Places in the MahabharataAnindita Basu (CC BY-NC-SA)

Mahabharata - World History Encyclopedia















































































































































 




Facebook





Membership
No ads








❯





 








Encyclopedia

Index
Timeline
Maps
Book Reviews
Weights & Measures
Translations



Education

Teaching Materials
Link Database
Quizzes
Primary Sources
Collections
School Subscription



Media

Media Library
Audio Articles
Podcast
Youtube Channel



Contribute

Submissions
Needed Content
Style Guide
Terms & Conditions
Volunteer
Donate



Shop

Books
Map Posters
Magazines
Wall Charts
Clothing



About

About
Our Team
Editorial Policy
Annual Reports
Membership
Contact



Login

Login
Register








Mahabharata






30 days left

Did you like this definition?







Subscrib

{'input': 'Name the characters of Mahabharata',
 'output': 'Some of the characters in the Mahabharata include Arjuna, the Kauravas (the 100 sons of King Dhritarashtra), Krishna, and the Pandavas (the five sons of Pandu).'}